# <span style="color:gray">ipyrad-analysis toolkit:</span> VCF2HDF5

Many genomic tools will write variant SNP calls to the VCF format (variant call format). This is a plain text file that stores variant calls relative to a reference genome in tabular format. It includes a lot of additional information about the quality of SNP calls, etc., but is not very easy to read or efficient to parse. To make analyses run a bit faster ipyrad uses a simplified format to store this information in the form of an HDF5 database. You can easily convert any VCF file to this HDF5 format using the `ipa.vcf_to_hdf5()` tool. 

This tool includes an added benefit of allowing you to enter an (optional) `ld_block_size` argument when creating the file which will write into the file information that can be used downstream by many other tools to subsample SNPs and perform bootstrap resampling in a way that reduces the effects of linkage among SNPs. If your data are assembled RAD data then the ld_block_size is not required, since we can simply use RAD loci as the linkage blocks. But if you want to combine reference-mapped RAD loci located nearby in the genome as being on the same linkage block then you can enter a value such as 50,000 to create 50Kb linkage block that will join many RAD loci together and sample only 1 SNP per block in each bootstrap replicate. If your data are not RAD data, e.g., whole genome data, then the ld_block_size argument will be required in order to  encode linkage information as discrete blocks into your database. 

### Required software

In [1]:
# conda install ipyrad -c bioconda 
# conda install htslib -c bioconda
# conda install bcftools -c bioconda

In [2]:
import ipyrad.analysis as ipa

### Pre-filter data from other programs (e.g., FreeBayes, GATK)

You can use the program `bcftools` to pre-filter your data to exclude indels and low quality SNPs. If you ran the `conda install` commands above then you will have all of the required tools installed. To achieve the format that ipyrad expects you will need to exclude indel containing SNPs (this may change in the future). Further quality filtering is optional. 

The example below reduced the size of a VCF data file from 29Gb to 80Mb! VCF contains a lot of information that you do not need to retain through all of your analyses. We will keep only the final genotype calls.  


In [ ]:
%%bash

# compress the VCF file if not already done (creates .vcf.gz)
bgzip data.vcf

# tabix index the compressed VCF (creates .vcf.gz.tbi)
tabix data.vcf.gz

# remove multi-allelic SNPs and INDELs and PIPE to next command
bcftools view -m2 -M2 -i'CIGAR="1X" & QUAL>30' data.vcf.gz -Ou | 

    # remove extra annotations/formatting info and save to new .vcf
    bcftools annotate -x FORMAT,INFO  > data.cleaned.vcf

### A peek at the reduced VCF file

In [ ]:
df.

In [10]:
import pandas as pd

# load the VCF as an datafram
df = pd.read_csv(
    "/home/deren/Documents/tetrad/data/Macaque-Chr1.clean.vcf",
    sep="\t", 
    skiprows=1000, 
    #chunksize=1000,
)

# show the first few lines
df.head()

,NC_018152.2,51273,.,G,A,280.482,..1,..2,GT,0/0,...,0/0.9,0/0.10,0/0.11,0/0.12,0/0.13,0/0.14,0/0.15,0/0.16,0/0.17,0/1.1
0,NC_018152.2,51292,.,A,G,16750.3000,.,.,GT,1/1,...,1/1,.,1/1,1/1,1/1,1/1,0/0,1/1,1/1,1/1
1,NC_018152.2,51349,.,A,G,628.5630,.,.,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
2,NC_018152.2,51351,.,C,T,943.3530,.,.,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3,NC_018152.2,51352,.,G,A,607.6810,.,.,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
4,NC_018152.2,51398,.,C,T,510.1200,.,.,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6093181,NC_018152.2,217458601,.,T,C,138.5900,.,.,GT,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
6093182,NC_018152.2,217458628,.,A,G,44.5791,.,.,GT,0/0,...,0/0,0/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
6093183,NC_018152.2,217458695,.,G,C,869.8200,.,.,GT,0/0,...,0/0,0/1,0/0,0/0,1/1,0/0,0/0,0/1,0/1,0/0
6093184,NC_018152.2,217458707,.,G,A,2386.0400,.,.,GT,0/0,...,0/0,1/1,1/1,0/0,1/1,0/0,0/0,1/1,1/1,0/1


In [ ]:
df.

### Required data files

In [11]:
data = "../../tests/ped-2019_outfiles/ped-2019.vcf"

data = "/home/deren/Documents/tetrad/Macaque-Chr1.cleaned.vcf.gz"


### Short tutorial

In [12]:
# init a conversion tool
self = ipa.vcf_to_hdf5(data=data, ld_block_size=200)

AttributeError: module 'ipyrad.analysis' has no attribute 'vcf_to_hdf5'

In [4]:
self.get_meta()

# run the conversion
#self.run()

VCF: 6094152 SNPs; 1 scaffolds


In [5]:
self.init_database()

In [6]:
self.build_matrix()

KeyboardInterrupt: 

In [12]:
# init a conversion tool
self = ipa.vcf_to_hdf5(data=data, ld_block_size=200)

# run the conversion
self.run()

Indexing VCF to HDF5 database file
VCF: 205405 snps; 39051 scaffolds
HDF5: 205405 snps; 39051 scaffolds; 39051 linkage group
SNP database written to ./analysis-vcf2hdf5/test.snps.hdf5


AttributeError: 'VCFtoHDF5' object has no attribute 'run'

In [3]:
# the new hdf5 database file
self.database

'./analysis-vcf2hdf5/test.snps.hdf5'

### Downstream analyses

In [4]:
tet = ipa.tetrad(
    name="test-vcf",
    data=self.database,
    nboots=10,
)

loading snps array [13 taxa x 205405 snps]
max unlinked SNPs per quartet (nloci): 39051
quartet sampler (full): 715 / 715


In [5]:
tet.run(auto=True, force=True, quiet=False)

Parallel connection | oud: 4 cores
initializing quartet sets database
[####################] 100% 0:00:05 | inferring full tree    
[####################] 100% 0:00:00 | bootstrap inference 1 
[####################] 100% 0:00:00 | bootstrap inference 2 
[####################] 100% 0:00:00 | bootstrap inference 3 
[####################] 100% 0:00:00 | bootstrap inference 4 
[####################] 100% 0:00:00 | bootstrap inference 5 
[####################] 100% 0:00:00 | bootstrap inference 6 
[####################] 100% 0:00:00 | bootstrap inference 7 
[####################] 100% 0:00:00 | bootstrap inference 8 
[####################] 100% 0:00:00 | bootstrap inference 9 
[####################] 100% 0:00:00 | bootstrap inference 10 


In [6]:
tet.trees

('tree', '/home/deren/Documents/ipyrad/tests/analysis-tetrad/test-vcf.tree')
('cons', '/home/deren/Documents/ipyrad/tests/analysis-tetrad/test-vcf.tree.cons')
('boots', '/home/deren/Documents/ipyrad/tests/analysis-tetrad/test-vcf.tree.boots')
('nhx', '/home/deren/Documents/ipyrad/tests/analysis-tetrad/test-vcf.tree.nhx')

In [11]:
toytree.tree(tet.trees.cons).root(wildcard="prz").draw(node_labels="support", use_edge_lengths=False);

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t675054117b7742a4b0b441e0865e577b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 360.0 275.0" width="360.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 35236_rex 30556_thamno 40578_rex 35855_rex 39618_rex 38362_rex 33413_thamno 30686_cyathophylla 29154_superba 41954_cyathophylloides 41478_cyathophylloides 33588_przewalskii 32082_przewalskii 50 100 30 100 90 100 100 100 100 100 100 100

In [13]:
ddd = "https://docs.google.com/a/columbia.edu/spreadsheets/d/e/2PACX-1vRvRWJNx_C6ovrXz-iQRRJ9UdkvpHLTotS13YI6VItkuO_fO94zRT4lM1wZx7q0z9V-FGp2CmS0rSqh/pub?gid=0&single=true&output=csv"

import pandas as pd
pd.read_csv(ddd)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 44, saw 4


In [4]:
io5 = h5py.File(self.database, 'r')
names = [i.decode() for i in io5["snps"].attrs["names"]]
samples = names
ntaxa = len(names)
nsnps = io5["snps"].shape[1]
print(
    "loading snps array [{} taxa x {} snps]".format(ntaxa, nsnps))
snpsmap = io5["snpsmap"][:]
snpsmap[:, 0] = snpsmap[:, 0] - 1
snpsmap[:, 1] = np.arange(snpsmap.shape[0])

loading snps array [10 taxa x 4776 snps]


In [5]:
snpsmap.astype(np.int32)

array([[      0,       0,    5029,       1,       0],
       [      0,       1,    5049,       1,       1],
       [      0,       2,    5098,       1,       2],
       ...,
       [   1254,    4773, 2737085,       1,    4773],
       [   1254,    4774, 2737089,       1,    4774],
       [   1254,    4775, 2737098,       1,    4775]], dtype=int32)

In [5]:
from tetrad.tetrad import *
from tetrad.jitted import jget_spans

In [8]:
io5 = h5py.File("ped-2019_outfiles/ped-2019.snps.hdf5", 'r')
names = [i.decode() for i in io5["snps"].attrs["names"]]
samples = names
ntaxa = len(names)
nsnps = io5["snps"].shape[1]
print(
    "loading snps array [{} taxa x {} snps]".format(ntaxa, nsnps))
snpsmap = io5["snpsmap"][:]
snpsmap[:, 0] = snpsmap[:, 0] - 1
snpsmap[:, 1] = np.arange(snpsmap.shape[0])

loading snps array [13 taxa x 205405 snps]


In [9]:
snpsmap

array([[     1,      0,      1,      0,      1],
       [     1,      1,     31,      0,      2],
       [     1,      2,     32,      0,      3],
       ...,
       [ 40795, 205402,     40,      0, 205403],
       [ 40795, 205403,     41,      0, 205404],
       [ 40795, 205404,     46,      0, 205405]], dtype=uint32)

In [12]:
jget_spans(snpsmap)

array([[     0,      5],
       [     5,     15],
       [    15,     22],
       ...,
       [205378, 205391],
       [205391, 205397],
       [205397, 205406]])

In [ ]:
maparr = snpsmap[:, :2]
sidx = 0
locs = np.unique(maparr[:, 0])
nlocs = locs.size
spans = np.zeros((nlocs, 2), np.int64)

### TODO:
sit down with a notebook and figure this out. I think current problem requires every locus to have variants. 

In [58]:
sidx = maparr[0, 0]
locs = np.unique(maparr[:, 0])
nlocs = locs.size
spans = np.zeros((nlocs, 2), np.int64)

lidx = 0
for idx in range(maparr.shape[0]):
    
    eidx = maparr[idx, 0]
    #print(eidx, sidx, lidx)
    
    if eidx != sidx:
        # the first value entered
        if not lidx:
            spans[lidx] = np.array((0, idx))

        # all other values
        else:
            spans[lidx] = spans[lidx - 1, 1], idx

        lidx += 1
        sidx = locs[lidx]
            
    spans[-1] = np.array((spans[-2, 1], maparr[-1, -1] + 1))

print(spans)

[[     0      5]
 [     5     15]
 [    15     22]
 ...
 [205378 205391]
 [205391 205397]
 [205397 205405]]


In [66]:
maparr[204397:205405, :]

array([[ 40600, 204397],
       [ 40601, 204398],
       [ 40601, 204399],
       ...,
       [ 40795, 205402],
       [ 40795, 205403],
       [ 40795, 205404]], dtype=uint32)

In [31]:
#@njit()
def jget_spans(maparr):
    """ 
    Returns array with span distances for each locus in original seqarray. 
    This is much faster than pandas or numpy queries.
    [ 0, 33],
    [33, 47],
    [47, 51], ...
    """
    sidx = 0
    locs = np.unique(maparr[:, 0])
    nlocs = locs.size
    spans = np.zeros((nlocs, 2), np.int64)

    lidx = 0
    # advance over all snp rows 
    for idx in range(maparr.shape[0]):
        
        # get locus id at this row 0, 0, 0, 0
        eidx = maparr[idx, 0]
        
        # if locus id is not sidx
        if eidx != sidx:
            try:

                if lidx:
                    spans[lidx] = spans[lidx - 1, 1], idx
                else:
                    spans[lidx] = np.array((0, idx))
                lidx += 1
                sidx = locs[lidx]
            except IndexError:
                pass

    # final end span
    spans[-1] = np.array((spans[-2, 1], maparr[-1, -1] + 1))
    return spans


jget_spans(maparr)

array([[     0,      0],
       [     0,      1],
       [     1,      2],
       ...,
       [ 39047,  39048],
       [ 39048,  39049],
       [ 39049, 205405]])

In [38]:
snpsmap[:25, :2]

array([[ 1,  0],
       [ 1,  1],
       [ 1,  2],
       [ 1,  3],
       [ 1,  4],
       [ 3,  5],
       [ 3,  6],
       [ 3,  7],
       [ 3,  8],
       [ 3,  9],
       [ 3, 10],
       [ 3, 11],
       [ 3, 12],
       [ 3, 13],
       [ 3, 14],
       [ 4, 15],
       [ 4, 16],
       [ 4, 17],
       [ 4, 18],
       [ 4, 19],
       [ 4, 20],
       [ 4, 21],
       [ 5, 22],
       [ 5, 23],
       [ 5, 24]], dtype=uint32)

In [ ]:
lidx = 0
for idx in range(maparr.shape[0]):
    
    eidx = maparr[idx, 0]
    
    if eidx != sidx:
        if lidx

In [ ]:
snpsmap[:, :2]

In [6]:
jget_spans(snpsmap[:, :2])

NameError: name 'snpsmap' is not defined

In [ ]:
io5 = h5py.File("ped-2019_outfiles/ped-2019.snps.hdf5", 'r')
names = [i.decode() for i in io5["snps"].attrs["names"]]
samples = names
ntaxa = len(names)
nsnps = io5["snps"].shape[1]
print(
    "loading snps array [{} taxa x {} snps]".format(ntaxa, nsnps))

# data base file to write the transformed array to
idb = h5py.File("/tmp/test.snps.hdf5", 'w')

# store maps info (enforced to 0-indexed?)
idb.create_dataset("bootsmap", (nsnps, 2), dtype=np.uint32)
snpsmap = io5["snpsmap"][:]
snpsmap[:, 0] = snpsmap[:, 0] - 1
snpsmap[:, 1] = np.arange(snpsmap.shape[0])
nloci = np.unique(snpsmap[:, 0]).size
idb["bootsmap"][:] = snpsmap[:, :2]

# store spans between loci
idb.create_dataset("spans", (nloci, 2), dtype=np.int64)
idb["spans"][:] = jget_spans(snpsmap[:, :2])

# store snps info
idb.create_dataset("seqarr", (ntaxa, nsnps), dtype=np.uint8)
tmpseq = io5["snps"][:].astype(np.uint8)
tmpseq[tmpseq == 45] = 78
tmpseq[tmpseq == 95] = 78
idb["seqarr"][:] = tmpseq

# boot samples: resolve ambigs and convert CATG bases to matrix indices
idb.create_dataset("bootsarr", (ntaxa, nsnps), dtype=np.uint8)
tmpseq = resolve_ambigs(tmpseq)       
tmpseq[tmpseq == 65] = 0
tmpseq[tmpseq == 67] = 1
tmpseq[tmpseq == 71] = 2
tmpseq[tmpseq == 84] = 3
idb["bootsarr"][:] = tmpseq

# report 
print(
    "max unlinked SNPs per quartet (nloci): {}"
    .format(nloci))

# cleanup
io5.close()
idb.close()
del tmpseq
del snpsmap

In [ ]:
io5 = h5py.File(self.database, 'r') 


In [ ]:
names = [i.decode() for i in io5["snps"].attrs["names"]]
ntaxa = len(names)
nsnps = io5["snps"].shape[1]
(ntaxa, nsnps), names

In [ ]:
io5.close()

In [ ]:
self.database

In [ ]:
#self.df["#CHROM"].factorize()

import tetrad

In [ ]:
from tetrad.tetrad import Tetrad

In [ ]:
import ipyrad.analysis as ipa
ipa.tetrad(
    name='test-tmp',
    data="/tmp/tmp-227.snps.hdf5",
    workdir="/tmp/test-tetrad",
)

In [ ]:
# with h5py.File(self.database, 'r') as io5:
#     print(io5[""][:])

In [ ]:
ldsize = 1000
lidx = 0
self.df[["#CHROM", "POS"]]#.groupby("#CHROM")

In [ ]:
df = pd.DataFrame({
    "#CHROM": ["A"] * 10 + ["B"] * 6,
    "POS": np.concatenate([
        np.linspace(1000, 5000, 10).astype(int),
        np.linspace(10000, 15000, 6).astype(int),
    ])
})

df

In [ ]:
idx = 0
for _, scaff in df.groupby("#CHROM"):
    df.loc[scaff.index, "#CHROM"] = idx
    idx += 1
    #print(scaff.index)

In [ ]:
df

In [ ]:
np.arange(10) + 10

In [ ]:
def chunker(df, ldsize=1000, lidx=0):
    
    # list to store chunks of scaffold
    chunks = []
    
    # get starting position on this scaffold
    start_pos = df.iloc[0, 1]
    
    # get all rows up to the ldsize
    chunk = df[(df.POS >= start_pos) & (df.POS < start_pos + ldsize)]
    
    # set new scaffold idx on chunk
    chunk.CHROM = lidx
    
    # add new counter for chunk snpsidx
    chunk["snpidx"] = range(chunk.shape[0])
    
    print(chunk)
    
    
chunker(df)
    

In [ ]:
self = conv

# load vcf as dataframe (TODO: chunk for large files)
self.df = pd.read_csv(self.vcf, sep="\t", skiprows=self.hlines)

# get ref and alt alleles as a string series
refalt = (self.df.REF + self.df.ALT).apply(str.replace, args=(",", ""))

# genos array to fill from geno indices and refalt
genos = np.zeros((self.nsnps, self.nsamples, 2), dtype="u1")
snps = np.zeros((self.nsnps, self.nsamples), dtype="S1")

print(genos.shape, refalt.shape)

# iterate over samples indices
for sidx in range(self.nsamples):

    # store geno calls for this sample (+9 goes to geno columns in vcf)
    glist = self.df.iloc[:, sidx + 9].apply(get_genos).apply(sorted)
    genos[:, sidx, :] = pd.concat([
        glist.apply(lambda x: x[0]),
        glist.apply(lambda x: x[1]),
    ], axis=1)

    # iterate over geno indices to get alleles
    for gidx in range(genos.shape[0]):

        # this sample's geno
        sgeno = genos[gidx, sidx]
        if sgeno[0] == 9:
            call = b"N"
            continue

        # convert to geno alleles
        call0 = refalt[gidx][sgeno[0]]
        call1 = refalt[gidx][sgeno[1]]

        # get ambiguity code
        if call0 != call1:
            call = TRANSFULL[(call0, call1)].encode()
        else:
            call = call0

        # store call
        snps[gidx, sidx] = call

# convert snps to uint8
snps = snps.astype("u1")
print(snps)

In [ ]:
genos[10, 7]

In [ ]:
snps.view("u1")

In [ ]:
refalt[782], [genos[782, 0, 1]]

In [ ]:
snps.fill("9")
snps

In [ ]:
genos[782, 0]

In [ ]:
genos = np.zeros((conv.nsnps, conv.nsamples, 2), dtype="u1")

genos.shape

In [ ]:
genos[:, 0, :].shape

In [ ]:
pd.concat([a.apply(lambda x: x[0]), a.apply(lambda x: x[1])], axis=1)

In [ ]:
a = conv.df.iloc[:, 9].apply(get_genos).apply(sorted)

In [ ]:
genos

In [ ]:
with h5py.File(conv.database, 'r') as io5:
    print(io5["genos"][:])

In [ ]:
refalt = (conv.df.REF + conv.df.ALT).apply(str.replace, args=(",", ""))

# genos array to fill from geno indices and refalt
genos = np.zeros((conv.nsamples, conv.nsnps, 2), dtype="u1")
snps = np.zeros((conv.nsamples, conv.nsnps), dtype="u4")        

In [ ]:
sorted((1, 0))

In [ ]:
refalt[0]

In [ ]:
def get_geno(gstr):
    return gstr[0], gstr[2]


conv.df.iloc[:, 9].apply(get_geno).apply(sorted)

In [ ]:
conv.df.ALT.values.astype("S1").view("u1")

In [ ]:
for i in conv.df[["#CHROM", "POS"]].groupby("#CHROM"):
    print(i)

In [ ]:
conv.vcf

In [ ]:
%%timeit
arr = pd.read_csv(conv.vcf, skiprows=conv.hlines, sep="\t")
#ser = arr["#CHROM"]

In [ ]:
with open(conv.vcf) as infile:
    while 1:
        dat = infile.readline()
        
    

In [ ]:
ser.factorize()[0]

### Current file

In [ ]:
print(io5["snpsmap"][-10:])

In [ ]:
with h5py.File("./test2-denovo_outfiles/test2-denovo.snps.hdf5") as io5:
    print(io5.keys())
    print(io5["snpsmap"][-10:])

In [ ]:
with h5py.File("./test2_outfiles/test2.snps.hdf5") as io5:
    print(io5.keys())
    print(io5["genos"])
    #print(io5['snps'])
    #print(io5["genos"][:10])
    

In [ ]:
io5 = h5py.File("./test2_outfiles//test2.snps.hdf5")

In [ ]:
import numpy as np

In [ ]:
np.zeros(10, dtype="i8")